<a href="https://colab.research.google.com/github/rahiakela/genai-research-and-practice/blob/main/generative-ai-with-langchain/02_customer_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

In [ ]:
!pip install langchain openai tiktoken accelerate cohere --quiet

In [3]:
from huggingface_hub import list_models
from langchain import HuggingFaceHub

from transformers import pipeline

from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

##Sentiment Model

In [7]:
def list_most_popular(task: str):
  for rank, model in enumerate(list_models(filter=task, sort="downloads", direction=-1)):
    if rank == 5:
      break
    print(f"{model.id}, {model.downloads}\n")

# We can list the 5 most downloaded models on Hugging Face Hub for text classification
list_most_popular("text-classification")

cardiffnlp/twitter-roberta-base-sentiment-latest, 47328119

lxyuan/distilbert-base-multilingual-cased-sentiments-student, 10379763

cardiffnlp/twitter-roberta-base-irony, 10217088

mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis, 10034515

SamLowe/roberta-base-go_emotions, 9812542



In [ ]:
# Let’s see what our sentiment model has to say
customer_email = """
I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you. I am writing to pour my heart out about the recent unfortunate experience I had with one of your coffee machines that arrived ominously broken, evoking a profound sense of disbelief and despair.
To set the scene, let me paint you a picture of the moment I anxiously unwrapped the box containing my highly anticipated coffee machine. The blatant excitement coursing through my veins could rival the vigorous flow of coffee through its finest espresso artistry. However, what I discovered within broke not only my spirit but also any semblance of confidence I had placed in your esteemed brand.
Imagine, if you can, the utter shock and disbelief that took hold of me as I laid eyes on a disheveled and mangled coffee machine. Its once elegant exterior was marred by the scars of travel, resembling a war-torn soldier who had fought valiantly on the fields of some espresso battlefield. This heartbreaking display of negligence shattered my dreams of indulging in daily coffee perfection, leaving me emotionally distraught and inconsolable
"""

sentiment_model = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)

In [9]:
print(sentiment_model(customer_email))

[{'label': 'LABEL_0', 'score': 0.5824514031410217}]


In [10]:
print(sentiment_model("I am so angry and sad, I want to kill myself!"))

[{'label': 'LABEL_0', 'score': 0.9788626432418823}]


In [11]:
print(sentiment_model("I am elated, I am so happy, this is the best thing that ever happened to me!"))

[{'label': 'LABEL_2', 'score': 0.9926880598068237}]


In [12]:
print(sentiment_model("I don't care. I guess it's ok, or not, I couldn't care one way or the other"))

[{'label': 'LABEL_1', 'score': 0.5958544611930847}]


##Summarization model

In [20]:
# Let’s execute the summarization model remotely on a server
summarizer = HuggingFaceHub(
    repo_id="facebook/bart-large-cnn",
    model_kwargs={"temperature": 0, "max_length": 180}
)

In [21]:
def summarize(llm, text)-> str:
  return llm(f"Summarize this: {text}")

In [22]:
summarize(summarizer, customer_email)

'"I am writing to pour my heart out about the recent unfortunate experience I had with one of your coffee machines that arrived ominously broken," writes the customer. "Imagine, if you can, the utter shock and disbelief that took hold of me as I laid eyes on a disheveled and mangled coffee machine," he writes. "This heartbreaking display of negligence shattered my dreams of indulging in daily coffee perfection," he adds.'

##Customer Service Assistance

In [23]:
template = """
Given this text, decide what is the issue the customer is concerned about. Valid categories are these:
* product issues
* delivery problems
* missing or late orders
* wrong product
* cancellation request
* refund or exchange
* bad support experience
* no clear reason to be upset

Text: {email}
Category:
"""

prompt = PromptTemplate(template=template, input_variables=["email"])
llm = VertexAI()
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

ValidationError: 1 validation error for VertexAI
__root__
  Unable to find your project. Please provide a project ID by:
- Passing a constructor argument
- Using vertexai.init()
- Setting project using 'gcloud config set project my-project'
- Setting a GCP environment variable
- To create a Google Cloud project, please follow guidance at https://developers.google.com/workspace/guides/create-project (type=value_error)